In [1]:
import numpy as np
import pandas as pd
import os 
import cv2 
import matplotlib.pyplot as plt
import torch
import torchvision

In [2]:
import torch 
import sys

sys.path.insert(0,'../input/BlazeFaceRepo/')

import blazeface

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
blazeface = blazeface.BlazeFace().to(device)
blazeface.load_weights("../input/BlazeFaceRepo/blazeface.pth")
blazeface.load_anchors("../input/BlazeFaceRepo/anchors.npy")

In [4]:
'''
each data point contains FRAME LENGTH consecutive frames
each data point is sampled every SAMPLE_LENGTH frames 
'''
def get_data_frame(filename, label, frame_length=10, sample_length=3, size=(128,128)) :
    cap = cv2.VideoCapture(filename)
    length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_count = 0
    data_points = []
    labels = []
    empty = True
    data_point = None
    while cap.isOpened() :
        modular = frame_count % sample_length
        ret, frame = cap.read()
        if ret : 
            if modular == 0 :
                '''
                the selected frame go through face detection 
                the face is cropped out and resized
                '''
                origin_h, origin_w = frame.shape[:2]
                frame_ = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frame_ = cv2.resize(frame_, (128,128))
                faces = blazeface.predict_on_image(frame_)
                '''
                if no face is found, the adjacent frame is taken instead
                '''
                if len(faces) == 0 :
                    continue
                
                faces = faces[(faces[:,0]-faces[:,2]).argsort()]
                x_tl, y_tl, x_br, y_br = faces[0,:4] 
                x_tl = int(x_tl * origin_w)
                x_br = int(x_br * origin_w)
                y_tl = int(y_tl * origin_h)
                y_br = int(y_br * origin_h)
                x_tl = x_tl if x_tl > 0 else 0 
                y_tl = y_tl if y_tl > 0 else 0 
                x_br = x_br if x_br < origin_w else origin_w - 1
                y_br = y_br if y_br < origin_h else origin_h - 1
                try :
                    face = cv2.resize(frame[y_tl:y_br, x_tl:x_br, :], size)
                    face = np.expand_dims(face, axis=0)
                except Exception as e :
                    print(e)
                    plt.imshow(frame)
                    plt.show()
                    print(frame.shape)
                    print(frame[y_tl:y_br, x_tl:x_br, :])
                    print((x_tl, y_tl, x_br, y_br))
                    continue
                if empty :
                    data_point = face
                    empty = False
                else :
                    data_point = np.vstack([data_point, face])
                    if data_point.shape[0] == frame_length :
                        data_points.append(np.expand_dims(data_point,axis=0))
                        labels.append(label)
                        empty = True
        else :
            break
        frame_count += 1
    cap.release()
    return data_points, labels

In [5]:
from tqdm import tqdm

FOLD1_1 = '../input/drowsiness-dataset-fold1-1/Fold1_part1'
ids = os.listdir('../input/drowsiness-dataset-fold1-1/Fold1_part1')
data_list = []
label_list = []
frame_list = []

for idx in ids :
    if idx != '01' :
        folder = os.path.join(FOLD1_1, idx)
        files = os.listdir(folder)
        for file in tqdm(files) :
            l = int(file[:-4])
            datas, labels= get_data_frame(os.path.join(folder, file), l)
            data_list += datas
            label_list += labels
test_list = []
test_label_list = []
folder = os.path.join(FOLD1_1, '01')
files = os.listdir(folder)
for file in tqdm(files) :
    l = int(file[:-4])
    datas, labels= get_data_frame(os.path.join(folder, file), l)
    test_list += datas
    test_label_list += labels

100%|██████████| 3/3 [12:57<00:00, 259.30s/it]


In [6]:
data = np.vstack(data_list)
label = np.array(label_list)
test = np.vstack(test_list)
test_label = np.array(test_label_list)

np.save('./data.npy', data)
np.save('./label.npy', label)
np.save('./test.npy', test)
np.save('./test_label.npy', test_label)

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [7]:
perm_test = np.random.permuation(test.shape[0])[:500]
perm_train = np.random.permuation(data.shape[0])[:5000]
data, label = data[perm_train], label[perm_train]
test, test_label = test[perm_test], test_label[perm_test]
np.save('./data_5k.npy', data)
np.save('./label_5k.npy', label)
np.save('./test_500.npy', test)
np.save('./test_label_500.npy', test_label)

AttributeError: module 'numpy.random' has no attribute 'permuation'